# Optimal power flow model in Julia

In [1]:
import Pkg

# activate parent environment
Pkg.activate(normpath(joinpath(@__DIR__, ".")))
Pkg.resolve()
Pkg.instantiate()
Pkg.status()

using CSV
using DataFrames

using Revise
using OptHP

  Activating project at `c:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort`
  No Changes to `C:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort\Project.toml`
  No Changes to `C:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort\Manifest.toml`
Precompiling project...
  ✓ OptHP
  1 dependency successfully precompiled in 7 seconds. 69 already precompiled.

Status `C:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort\Project.toml`
  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.7.0
  [2d871509] OptHP v1.0.0-DEV `OptHP`
  [295af30f] Revise v3.6.4

## Load data

In [53]:
network = CSV.read("data/network.csv", DataFrame)

# correct StarNode and EndNode for 1 based indexing
network.StartNode = network.StartNode .+ 1
network.EndNode = network.EndNode .+ 1

first(network, 5)

Row,StartNode,EndNode,Length,cable,Inom,R,X
,Int64,Int64,Float64,String7,Float64,Float64,Float64
1,48,16,22.21,50Al,0.12528,0.0142366,0.00188785
2,50,41,15.57,95Al,0.18386,0.0049824,0.00127674
3,27,42,18.26,95Al,0.18386,0.0058432,0.00149732
4,12,13,6.71,95Al,0.18386,0.0021472,0.00055022
5,13,28,6.52,95Al,0.18386,0.0020864,0.00053464


In [54]:
connections = CSV.read("data/user_connect.csv", DataFrame; delim=";")

# correct for 1 based indexing
connections.Node = connections.Node .+ 1

first(connections, 5)

Row,Column1,Name,Node.ID,Node.Unom,Length,Type short,Connection value,load,PV,HP,Node
,Int64,Int64,Int64,Int64,String7,String15,String7,Int64,String7,Int64,Int64
1,1,871694840006284337,46559968,230,"21,75",XLPE 4x6Cu,3x25 A,7000,4,1,51
2,2,871694840006288793,46563894,230,"2,22",XLPE 4x6Cu,3x25 A,7000,6,0,52
3,3,871694840006284108,11195660,230,"10,4",PVC 4x16Al,1x35 A,2000,"2,2",0,53
4,4,871694840006284115,11195660,230,"10,4",PVC 4x16Al,1x35 A,500,"2,2",0,54
5,5,871694840006284092,2573422,230,"7,33",PVC 4x16Al,1x35 A,2000,"2,2",0,55


In [ ]:
loads_real = CSV.read("data/UserPower.csv", DataFrame)
loads_reactive = CSV.read("data/UserReactivePower.csv", DataFrame)
# convert 7/1/2023 12:00 AM to datetime
loads_real.time = Date.(loads_real.time, "m/d/yyyy H:M AM")

Row,Column1,time,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37
,Int64,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0,7/1/2023 12:00 AM,-0.000976027,-0.000966499,-0.000326979,-7.78134e-5,-0.000304578,-0.000224549,-0.000707056,-0.000827133,-0.00106905,-0.000311715,-7.54402e-5,-0.00036558,-0.000140259,-0.000160978,-0.000723012,-0.000236992,-0.00055929,-0.00165917,-0.000625475,-0.000558187,-0.000411753,-0.00139921,-0.000329897,-0.000400155,-0.00101174,-0.001642,-0.000564799,-0.000298986,-0.000713016,-0.000325963,-0.000325851,-0.000474203,-0.000890957,-0.000708738,-0.000453405,-0.000910606,-0.0
2,1,7/1/2023 12:15 AM,0.0106061,0.0100529,0.0026215,0.000744874,0.00268222,0.00193239,0.00606037,0.0074748,0.00963479,0.00298592,0.000767212,0.00321348,0.00137063,0.00141256,0.00672597,0.00200243,0.00513401,0.0136152,0.00569069,0.00472781,0.00335423,0.0138645,0.00259936,0.00370932,0.00971926,0.0151974,0.00512517,0.00273394,0.00690411,0.00279824,0.00268846,0.00431241,0.00900718,0.00575147,0.00436711,0.00839781,0.0
3,2,7/1/2023 12:30 AM,-0.00108302,-0.00108993,-0.000289105,-8.05129e-5,-0.000313582,-0.000228986,-0.000769064,-0.000868664,-0.0010676,-0.000309442,-8.80597e-5,-0.000411046,-0.000170757,-0.000169152,-0.00086,-0.00025217,-0.000614113,-0.00168496,-0.000666674,-0.00051133,-0.000428647,-0.0017357,-0.000295446,-0.000385634,-0.00115144,-0.00161289,-0.000555897,-0.000337292,-0.000880754,-0.000313174,-0.000306404,-0.000463878,-0.000958423,-0.000786564,-0.000507658,-0.000961887,-0.0
4,3,7/1/2023 12:45 AM,-0.00101928,-0.0011714,-0.000334496,-8.11756e-5,-0.000323302,-0.000250492,-0.000657047,-0.000709221,-0.00113771,-0.00031237,-8.37059e-5,-0.000401931,-0.000170925,-0.00014955,-0.000750459,-0.000229597,-0.000540842,-0.00147015,-0.000577555,-0.000502747,-0.000364227,-0.00158832,-0.000317853,-0.000359276,-0.00104949,-0.00170677,-0.000559206,-0.000302791,-0.000858837,-0.000340213,-0.000334116,-0.000461295,-0.000856842,-0.000669867,-0.000433632,-0.000941337,-0.0
5,4,7/1/2023 1:00 AM,0.00219411,0.00206907,0.000605413,0.000159578,0.00053021,0.000466835,0.00140087,0.00136803,0.0020747,0.000558547,0.000134342,0.000699626,0.000290488,0.00031443,0.00147617,0.000472594,0.000988679,0.00279557,0.00115383,0.000993602,0.000760146,0.00287413,0.000625225,0.000798681,0.00201701,0.00297571,0.000923752,0.000568639,0.00151264,0.000617864,0.000605775,0.00085955,0.00158486,0.00129725,0.00087325,0.00175833,0.0
6,5,7/1/2023 1:15 AM,0.00473608,0.00515323,0.00146932,0.000346422,0.00134609,0.00110865,0.0034237,0.00404778,0.00489715,0.00150998,0.000360865,0.00193591,0.000807714,0.000666508,0.00350606,0.00109486,0.0025362,0.00734964,0.00302404,0.00274401,0.00170672,0.00693266,0.00161878,0.00167125,0.00559811,0.00738281,0.00242499,0.00154027,0.00396681,0.00144303,0.00132996,0.00204094,0.00420513,0.00345537,0.00200105,0.00441856,0.0
7,6,7/1/2023 1:30 AM,-0.000931039,-0.00109069,-0.000299908,-6.96579e-5,-0.000300925,-0.000228754,-0.000591847,-0.000652547,-0.000997238,-0.000257871,-7.29876e-5,-0.000334582,-0.000130401,-0.000153822,-0.000773582,-0.000208816,-0.000474194,-0.00150397,-0.000613631,-0.000509962,-0.000369973,-0.00156316,-0.000270088,-0.000345665,-0.000984499,-0.00131266,-0.00047799,-0.000280344,-0.000743763,-0.000268285,-0.000301998,-0.000447333,-0.000776525,-0.00060338,-0.000448861,-0.000858702,-0.0
8,7,7/1/2023 1:45 AM,-0.00100378,-0.00100561,-0.000286975,-7.89301e-5,-0.000307743,-0.000221306,-0.000620535,-0.000692985,-0.00095657,-0.000291586,-7.4978e-5,-0.000358756,-0.000152694,-0.000135936,-0.000786127,-0.000209062,-0.000482753,-0.00144363,-0.000551019,-0.00049681,-0.000386615,-0.00153346,-0.000271161,-0.000372212,-0.00113563,-0.00160842,-0.000489567,

## Build graph model

In [55]:
# Step 1: Identify all unique nodes
nodes = 0:maximum(vcat(network.EndNode, network.StartNode))
node_indices = Dict(node => i for (i, node) in enumerate(nodes))  # Map nodes to indices

# Step 2: Initialize an adjacency matrix
n = length(nodes)
adjacency_matrix = zeros(Int, n, n)

# Step 3: Populate the matrix with connections
for (start_node, end_node) in zip(network.StartNode, network.EndNode)
    j = node_indices[start_node]
    i = node_indices[end_node]
    adjacency_matrix[i, j] = 1
end

# this gives:
# --- StartNode ---
#   0, 1, 2, 3, ...
# 0 1, 0, 0, 0, ... 
# 1 0, 0, 1, 0, ...
# 2 0, 0, 0, 1, ...
df_adj = DataFrame(adjacency_matrix, string.(nodes))

# save as CSV
CSV.write("data/adjacency_matrix.csv", df_adj, delim=",")

"data/adjacency_matrix.csv"

## Construct GEC

In [72]:
model = GEC(network=network, connections=connections)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Set parameter Username
Set parameter LicenseID to value 2599292
Academic license - for non-commercial use only - expires 2025-12-13


(2-dimensional DenseAxisArray{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, JuMP.ScalarShape},2,...} with index sets:
    Dimension 1, 1:96
    Dimension 2, 1:86
And data, a 96×86 Matrix{JuMP.ConstraintRef{JuMP.Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, JuMP.ScalarShape}}:
 0.02847322 PLine[1,1] + 0.0037757 QLine[1,1] + VoltSquare[16,1] - VoltSquare[48,1] - 0.00020624504191460002 CurrentSquare[1,1] == 0       …  0.0258232 PLine[86,1] + 0.00129792 QLine[86,1] - VoltSquare[29,1] + VoltSquare[87,1] - 0.0001671305636416 CurrentSquare[86,1] == 0
 0.02847322 PLine[1,2] + 0.0037757 QLine[1,2] + VoltSquare[16,2] - VoltSquare[48,2] - 0.00020624504191460002 CurrentSquare[1,2] == 0          0.0258232 PLine[86,2] + 0.00129792 QLine[86,2] - VoltSquare[29,2] + VoltSquare[87,2] - 0.0001671305636416 CurrentSquare[86,2] ==